# Lab: Descriptor

Write a (non-data) descriptor that will allow us to calculate an attribute value the *first* time it is loaded, and cache it for subsequent loads.

In [1]:
class NonDataDescriptor:
    def __init__(self, getter):
        self._getter = getter
        self._attrname = None
        
    def __set_name__(self, cls, name):
        self._attrname = name
        
    def __get__(self, instance, type_):
        #print('Calling the descriptor')
        if instance is None:
            return self
        value = self._getter(instance)
        #setattr(instance, self._attrname, value)
        instance.__dict__[self._attrname] = value
        return value

In [2]:
class MyClass:
    
    @NonDataDescriptor
    def non_data_descriptor(self):
        print('Calculate non_data_descriptor!')
        return 'non_data_descriptor'

In [3]:
obj = MyClass()
obj.non_data_descriptor

Calculate non_data_descriptor!


'non_data_descriptor'

In [4]:
obj.non_data_descriptor

'non_data_descriptor'

Implement the descriptor above as a *data* descriptor:

In [5]:
class DataDescriptor():
    missing = object()
    
    def __init__(self, getter):
        self._getter = getter
        self._attrname = None
        
    def __set_name__(self, cls, name):
        self._attrname = name
        
    def __get__(self, instance, type):        
        # print('Calling the descriptor', self, instance, type)
        if instance is None:
            return self
        # Check to see if the value is in the instance dict
        value = instance.__dict__.get(self._attrname, self.missing)
        if value is self.missing:
            value = self._getter(instance)
            instance.__dict__[self._attrname] = value
            # setattr(instance, self._attrname, value)  will not work here. why?
        return value
    
    def __set__(self, instance, value):
        raise TypeError

In [6]:
class MyClass:
    
    @NonDataDescriptor
    def non_data_descriptor(self):
        print('Calculate non_data_descriptor!')
        return 'non_data_descriptor'
    
    @DataDescriptor
    def data_descriptor(self):
        print('Calculate data_descriptor!')
        return 'data_descriptor'

In [7]:
obj = MyClass()
obj.data_descriptor

Calculate data_descriptor!


'data_descriptor'

In [8]:
obj.data_descriptor

'data_descriptor'

In [9]:
obj.data_descriptor = 5

TypeError: 

Evaluate the performance of the data descriptor version versus the non-data-descriptor version.

(You can use the `%timeit` or `%%timeit` jupyter magic to get the timing of a line (or cell) of code.)

Data descriptor

In [10]:
obj.non_data_descriptor

Calculate non_data_descriptor!


'non_data_descriptor'

In [11]:
%timeit obj.data_descriptor

270 ns ± 8.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Non-data descriptor

In [12]:
%timeit obj.non_data_descriptor

63.1 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
